In [33]:
import sys 
import os 
import random 
import pandas as pd 
import glob 
from pydub import AudioSegment
from pydub.generators import WhiteNoise
sys.path.append('/root')
from nas_tools.utils.utils import *

### 下载数据
1. tts数据

In [3]:
download_file(
    remote_filepath='boweihan/data/synthesis_data/tts_data/complete_tts_table.csv'
    ,local_filepath='/root/nuna_simulation_test/data/tts_data/complete_tts_table.csv'
)

文件已下载并保存为: /root/nuna_simulation_test/data/tts_data/complete_tts_table.csv


0

In [5]:
all_tts_data = pd.read_csv('/root/nuna_simulation_test/data/tts_data/complete_tts_table.csv') 
all_tts_data.head(2)

,tts_model,voice_id,model_voice_id,style,emotion_value,emotion_id,conversation_id,conversation_index,voice_nas_path
0,murf,voicee042859e-168d-47cc-ac6e-7a6cb29f5ed5,en-US-cooper,Disappointment,Disappointment,11,dfd62aaa-1cae-40ba-aca8-f237fefcac71,0,boweihan/data/synthesis_data/tts_data/murf_tts...
1,murf,voice18e00ace-fcd1-462b-949e-c3f857996ae8,en-US-charles,Sobbing,Sadness,6,dfd62aaa-1cae-40ba-aca8-f237fefcac71,1,boweihan/data/synthesis_data/tts_data/murf_tts...


In [6]:
all_conversation_id_list = list(set(all_tts_data['conversation_id'].values.tolist()))
print(len(all_conversation_id_list))

6263


In [36]:
sampled_conversation_list = random.sample(all_conversation_id_list, 5)
sampled_conversation_list

['e8276765-7cc3-4351-b8b2-c015e72a1e80',
 'f885db12-4d1f-4df3-9284-ec9306614824',
 '45d03483-dc2f-4a14-b23f-1f4f39c9fb3f',
 'c0b80c67-3324-42af-99b6-2c9870c75205',
 '1d0b3361-f2bc-45df-9472-6fa278faee36']

In [ ]:
tts_data_dict = {} 
tts_local_folder_path = '/root/nuna_simulation_test/data/tts_data/conversation_turn_data/'
for conv_id in sampled_conversation_list:
    temp_data = all_tts_data[all_tts_data['conversation_id'] == conv_id] 
    conversation_index_list = temp_data['conversation_index'].values.tolist() 
    emotion_value_list = temp_data['emotion_value'].values.tolist()  
    emotion_id_list = temp_data['emotion_id'].values.tolist() 
    voice_nas_path_list = temp_data['voice_nas_path'].values.tolist() 
    user_all_emotion_dict = {}
    assistant_all_emotion_dict = {}
    for i in range(len(temp_data)): 
        conversation_index = conversation_index_list[i] 
        emotion_value = emotion_value_list[i] 
        emotion_id = emotion_id_list[i] 
        voice_nas_path = voice_nas_path_list[i]
        local_filepath = os.path.join(tts_local_folder_path, conv_id, voice_nas_path.split('/')[-1]) 
        # download_file(
        #     remote_filepath=voice_nas_path
        #     ,local_filepath=local_filepath
        # )
        if conv_id not in tts_data_dict:
            tts_data_dict[conv_id] = {}
        tts_data_dict[conv_id][conversation_index] = {
            'conv_id': conv_id 
            ,'emotion_value': emotion_value 
            ,'emotion_id': emotion_id 
            ,'voice_nas_path': voice_nas_path 
            ,'conversation_index': conversation_index 
            ,'local_filepath': local_filepath
        }
        if int(conversation_index) % 2 != 0:
            user_all_emotion_dict[emotion_id] = emotion_value 
        else: 
            assistant_all_emotion_dict[emotion_id] = emotion_value
    print(all_emotion_dict)
    break 

{2: 'Pleasure（Happy)', 10: 'Frustration', 4: 'Admiration'}


In [17]:
with open("./data/tts_data.json", "w", encoding="utf-8") as write_f:
    json.dump(tts_data_dict, write_f, ensure_ascii=False, indent=4)


### noise data

In [19]:
download_file(
    remote_filepath='chaoqun/audio_scene_label.csv'
    ,local_filepath=os.path.join(f'/root/nuna_simulation_test/data/noise_data/audio_scene_label.csv')
)

文件已下载并保存为: /root/nuna_simulation_test/data/noise_data/audio_scene_label.csv


0

In [20]:
audio_scene_data = pd.read_csv('/root/nuna_simulation_test/data/noise_data/audio_scene_label.csv')
audio_scene_data.head(2)

,audio_name,first_scene,second_scene
0,ATMs_1_001.wav,Banks and Financial Institutions,ATMs
1,ATMs_1_002.wav,Banks and Financial Institutions,ATMs


In [21]:
# 下载所有噪音切片数据，如果已下载则注释
# second_scene_list = list(set(audio_scene_data['second_scene'].values.tolist()))
# print(second_scene_list)
# for second_scene in second_scene_list: 
#     temp_data = audio_scene_data[audio_scene_data['second_scene'] == second_scene].sample(10) 
#     audio_name_list = temp_data['audio_name'].values.tolist() 
#     first_scene_list11 = temp_data['first_scene'].values.tolist()
#     second_scene_list11 = temp_data['second_scene'].values.tolist()
#     print(len(temp_data))
#     # break/ 
#     for i in range(len(temp_data)):
#         nas_file_path = os.path.join('chaoqun/scene_audio_from_youtube', first_scene_list11[i], second_scene_list11[i], audio_name_list[i]) 
#         print(nas_file_path)
#         download_file(
#             remote_filepath=nas_file_path
#             ,local_filepath=os.path.join(f'/root/nuna_simulation_test/data/noise_data/second_scene/{second_scene_list11[i]}/{audio_name_list[i]}')
#         )

### make origin voice data

In [24]:
noise_datapath = './data/noise_data/second_scene/' 
tts_datapath = './data/tts_data/conversation_turn_data'

In [25]:
def get_all_folders_from_path(folder_path):
    # 获取所有文件和文件夹名称
    all_items = os.listdir(folder_path)
    # 筛选出文件夹
    return [item for item in all_items if os.path.isdir(os.path.join(folder_path, item))]

all_scene_noise_folder_list = get_all_folders_from_path(noise_datapath)
print(all_scene_noise_folder_list)

all_tts_folder_list = get_all_folders_from_path(tts_datapath)
print(all_tts_folder_list)

['fitness studios', 'Bedroom', 'Churches', 'synagogues', 'clinics', 'kitchen', 'Post offices', 'Supermarkets', 'trails', 'town halls', 'Gas stations', 'train stations', 'Gyms', 'Bus stops', 'Restaurants', 'Banks', 'Public libraries', 'university libraries', 'mosques', 'university', 'factory', 'malls', 'Rec centers', 'School', 'living room', 'Parks', 'coffee shops', 'Office', 'ATMs', 'Hospitals']
['7aae8ae6-efed-4e26-8a94-3233dea7cc21', 'e9f6bbeb-5742-4efd-a80b-4798c9109b33', 'cc722536-b171-4118-80c8-9960974c043c', 'ef8a5858-8416-40c2-91e7-480da1934834', '9dd6082d-2058-4158-9bbe-0f8d209f4910']


In [30]:
def get_all_wav_files_from_folder_path(folder_path): 
    rst1 = glob.glob(f"{folder_path}/*.wav")
    rst2 = glob.glob(f"{folder_path}/*.WAV")
    return rst1 + rst2

def get_random_noise_voice_data(folder_path): 
    noise_folder_path = random.choice(all_scene_noise_folder_list) 
    all_scene_wav_files = get_all_wav_files_from_folder_path(os.path.join(folder_path, noise_folder_path)) 
    return random.choice(all_scene_wav_files)

get_random_noise_voice_data(noise_datapath)

'./data/noise_data/second_scene/Office/Office_1_127.wav'

In [31]:
def get_tts_human_voice_data(folder_path, output_file, min_pause=1000, max_pause=5000): 
    tts_folder_path = random.choice(all_tts_folder_list) 
    all_conversation_turn_files = get_all_wav_files_from_folder_path(os.path.join(folder_path, tts_folder_path)) 
    all_conversation_turn_file_dict = {}
    for filename in all_conversation_turn_files:
        wav_name = filename.split('/')[-1] 
        turn_cnt = int(wav_name.split('_')[-1].replace('.wav', '').replace('.WAV', '')) 
        all_conversation_turn_file_dict[turn_cnt] = filename
    sorted_all_conversation_turn_file_dict = dict(sorted(all_conversation_turn_file_dict.items(), key=lambda item: item[0], reverse=False))
    # print(sorted_all_conversation_turn_file_dict) 
    combined = AudioSegment.silent(duration=0) # 初始化空白音频段
    cnt = 0 
    for _, turn_audio_datapath in sorted_all_conversation_turn_file_dict.items(): 
        # print(turn_audio_datapath)
        segment = AudioSegment.from_wav(turn_audio_datapath) 
        combined += segment
        pause_duration = random.randint(min_pause, max_pause)
        silence = AudioSegment.silent(duration=pause_duration)
        combined += silence
        cnt += 1
        if cnt == 5:
            break 
    combined.export(output_file, format="wav")
    print(f"文件已保存至: {output_file}")

In [32]:
get_tts_human_voice_data(
    folder_path = tts_datapath
    ,output_file = './data/origin_voice_data/debug.wav'
)

文件已保存至: ./data/origin_voice_data/debug.wav
